Nama : **Fauzan Ahsanudin Alfikri**

Kelas : **DS-47-03**

NIM : **103052300003**

In [31]:
import os
import base64
import json
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import hashes, padding
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
import pandas as pd
from google.colab import files


In [32]:
def upload_file():
    print("📤 Silakan pilih file untuk diupload...")
    uploaded = files.upload()

    if not uploaded:
        return None

    filename = list(uploaded.keys())[0]
    print(f"✓ File berhasil diupload: {filename}")
    return filename


In [33]:
def derive_key(password: str, salt: bytes) -> bytes:
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    return kdf.derive(password.encode())


In [34]:
def encrypt_text(plaintext: str, password: str) -> dict:
    salt = os.urandom(16)
    iv = os.urandom(16)

    key = derive_key(password, salt)

    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(plaintext.encode()) + padder.finalize()

    cipher = Cipher(
        algorithms.AES(key),
        modes.CBC(iv),
        backend=default_backend()
    )
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()

    return {
        'ciphertext': base64.b64encode(ciphertext).decode('utf-8'),
        'iv': base64.b64encode(iv).decode('utf-8'),
        'salt': base64.b64encode(salt).decode('utf-8')
    }


In [35]:
def decrypt_text(encrypted_data: dict, password: str) -> str:
    ciphertext = base64.b64decode(encrypted_data['ciphertext'])
    iv = base64.b64decode(encrypted_data['iv'])
    salt = base64.b64decode(encrypted_data['salt'])

    key = derive_key(password, salt)

    cipher = Cipher(
        algorithms.AES(key),
        modes.CBC(iv),
        backend=default_backend()
    )
    decryptor = cipher.decryptor()
    padded_plaintext = decryptor.update(ciphertext) + decryptor.finalize()

    unpadder = padding.PKCS7(128).unpadder()
    plaintext = unpadder.update(padded_plaintext) + unpadder.finalize()

    return plaintext.decode('utf-8')


In [40]:
def encrypt_file(input_file: str, password: str) -> dict:

    file_size_mb = os.path.getsize(input_file) / (1024 * 1024)
    if file_size_mb > 1:
        raise ValueError(f"Ukuran file ({file_size_mb:.2f} MB) melebihi batas 1 MB")

    try:
        with open(input_file, 'rb') as f:
            file_bytes = f.read()

        base64_string = base64.b64encode(file_bytes).decode('utf-8')

        print(f"Mengenkripsi {input_file} ({len(file_bytes)} bytes)...")
        encrypted_data = encrypt_text(base64_string, password)
        print("✓ Enkripsi Base64 selesai.")

        return encrypted_data

    except FileNotFoundError:
        raise FileNotFoundError(f"File tidak ditemukan: {input_file}")
    except Exception as e:
        raise Exception(f"Error saat mengenkripsi file: {e}")

In [41]:
def decrypt_file(encrypted_data: dict, password: str, output_file: str):
    try:
        print("Mendekripsi data...")
        base64_string = decrypt_text(encrypted_data, password)
        print("✓ Dekripsi Base64 selesai.")

        file_bytes = base64.b64decode(base64_string)

        with open(output_file, 'wb') as f:
            f.write(file_bytes)

        print(f"✓ File berhasil didekripsi dan disimpan sebagai: {output_file}")

    except Exception as e:
        raise Exception(f"Error saat mendekripsi file: {e}")

In [38]:
def main():
    print("=" * 70)
    print("   APLIKASI KRIPTOGRAFI AES-256 CBC - Data Science Telkom")
    print("=" * 70)

    while True:
        print("\n" + "=" * 60)
        print("MENU: 1.Enkripsi Teks | 2.Dekripsi Teks")
        print("      3.Enkripsi File | 4.Dekripsi File | 0.Keluar")
        print("=" * 60)

        choice = input("\nPilih (0-4): ").strip()

        try:
            if choice == '1':
                plaintext = input("Teks yang akan dienkripsi: ")
                password = input("Password: ")

                if not plaintext or not password:
                    print("❌ Input tidak boleh kosong!")
                    continue

                encrypted = encrypt_text(plaintext, password)

                output_file = f"encrypted_text_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.json"
                with open(output_file, 'w') as f:
                    json.dump(encrypted, f, indent=2)

                print(f"\n✓ BERHASIL DIENKRIPSI!")
                print(f"File tersimpan: {output_file}")
                print(f"\nIsi file JSON:")
                print(f"  Ciphertext: {encrypted['ciphertext'][:60]}...")
                print(f"  IV: {encrypted['iv']}")
                print(f"  Salt: {encrypted['salt']}")

            elif choice == '2':
                print("\n📤 Upload file JSON hasil enkripsi...")
                json_file = upload_file()

                if not json_file:
                    print("❌ Upload dibatalkan!")
                    continue

                with open(json_file, 'r') as f:
                    encrypted_data = json.load(f)

                password = input("Password: ")
                if not password:
                    print("❌ Password tidak boleh kosong!")
                    continue

                plaintext = decrypt_text(encrypted_data, password)

                output_file = f"decrypted_text_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.txt"
                with open(output_file, 'w', encoding='utf-8') as f:
                    f.write(plaintext)

                print(f"\n✓ BERHASIL DIDEKRIPSI!")
                print(f"File tersimpan: {output_file}")
                print(f"\nHasil dekripsi:")
                print(f"  {plaintext}")

            elif choice == '3':
                print("\n📤 Upload file (Excel, CSV, dll - Maks 1MB) untuk dienkripsi...")
                input_file = upload_file()

                if not input_file:
                    print("❌ Upload dibatalkan!")
                    continue

                password = input("Password: ")
                if not password:
                    print("❌ Password tidak boleh kosong!")
                    continue

                try:
                    encrypted_data = encrypt_file(input_file, password)

                    output_file = f"{input_file}_encrypted.json"
                    metadata = {
                        'original_filename': input_file,
                        'encrypted_content': encrypted_data
                    }

                    with open(output_file, 'w') as f:
                        json.dump(metadata, f, indent=2)

                    print(f"\n✓ FILE BERHASIL DIENKRIPSI!")
                    print(f"File input: {input_file}")
                    print(f"File output: {output_file}")

                except ValueError as ve:
                    print(f"\n❌ GAGAL: {ve}")
                except Exception as e:
                    print(f"\n❌ Error saat enkripsi file: {e}")


            elif choice == '4':
                print("\n📤 Upload file JSON terenkripsi...")
                json_file = upload_file()

                if not json_file:
                    print("❌ Upload dibatalkan!")
                    continue

                with open(json_file, 'r') as f:
                    metadata = json.load(f)

                password = input("Password: ")
                if not password:
                    print("❌ Password tidak boleh kosong!")
                    continue

                encrypted_data = metadata['encrypted_content']
                original_filename = metadata['original_filename']
                output_file = f"decrypted_{original_filename}"

                decrypt_file(encrypted_data, password, output_file)

                print(f"\n✓ FILE BERHASIL DIDEKRIPSI!")
                print(f"File output disimpan sebagai: {output_file}")
                print(f"Nama file asli: {original_filename}")

            elif choice == '0':
                break
            else:
                print("❌ Pilihan tidak valid! Silakan pilih antara 0-4.")

        except Exception as e:
            print(f"\n❌ TERJADI ERROR: {str(e)}")
            print("💡 Tips: Pastikan password benar dan format file valid.")

In [39]:
main()

   APLIKASI KRIPTOGRAFI AES-256 CBC - Data Science Telkom

MENU: 1.Enkripsi Teks | 2.Dekripsi Teks
      3.Enkripsi File | 4.Dekripsi File | 0.Keluar

Pilih (0-4): 0
